# Import the data

In [1]:
import pandas as pd 
import numpy as np 

import statsmodels
import statsmodels.api as sm
import scipy.stats as stats

import matplotlib.pyplot as plt

# import the csv file with JUST the politicians post
comDB = pd.read_csv(r"/Users/tassan-mazzoccoarthur/Desktop/NETWORK SCIENCE (MOD. B)/1_Project/database/com_liwc.csv", sep='\t', engine='python')

df = pd.DataFrame(data=comDB)
df

,Origin_file_order,Site,p_id,dateCreated,p_politician,p_gender,p_GRUPPO_PE,p_LISTA,p_PARTITO,p_governo,...,p_Comma,p_Colon,p_SemiC,p_Qmark,p_Exclam,p_Dash,p_Quote,p_Apostro,p_Parenth,p_OtherP
0,30126,FB,96844400700_10157493758850701,2019-04-23T10:33:37Z,MARCELLO GEMMATO,M,PPE,FDI,FDI,opposizione,...,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00
1,30226,FB,96844400700_10157493758850701,2019-04-23T10:33:37Z,MARCELLO GEMMATO,M,PPE,FDI,FDI,opposizione,...,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00
2,60063,FB,96844400700_10157493758850701,2019-04-23T10:33:37Z,MARCELLO GEMMATO,M,PPE,FDI,FDI,opposizione,...,16.67,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00
3,60852,FB,96844400700_10157493758850701,2019-04-23T10:33:37Z,MARCELLO GEMMATO,M,PPE,FDI,FDI,opposizione,...,22.22,0.00,0.0,0.0,0.0,0.0,0.0,11.11,0.0,0.00
4,61557,FB,96844400700_10157493758850701,2019-04-23T10:33:37Z,MARCELLO GEMMATO,M,PPE,FDI,FDI,opposizione,...,22.22,0.00,0.0,0.0,0.0,0.0,0.0,11.11,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78170,70705,Twitter,1.11852926516164E+018,2019-04-17T14:58:52Z,MATTEO SALVINI,M,EFDD,LEGA,Lega,governo,...,6.82,2.27,0.0,0.0,0.0,0.0,0.0,6.82,0.0,6.82
78171,43288,Twitter,1.11846561220288E+018,2019-04-17T10:45:56Z,CÉCILE KYENGE,F,PSE,Siamo Europei,PD,opposizione,...,6.82,2.27,0.0,0.0,0.0,0.0,0.0,6.82,0.0,6.82
78172,70478,Twitter,1.11825025930045E+018,2019-04-16T20:30:12Z,MATTEO SALVINI,M,EFDD,LEGA,Lega,governo,...,3.12,6.25,0.0,0.0,0.0,0.0,0.0,0.00,0.0,9.38
78173,69430,Twitter,1.11824474991438E+018,2019-04-16T20:08:19Z,MATTEO SALVINI,M,EFDD,LEGA,Lega,governo,...,5.41,2.70,0.0,0.0,0.0,0.0,0.0,0.00,0.0,13.51


# Matrix creation

In [2]:
#Setting variables
size_df= len(df)
parties=df['p_PARTITO'].unique().tolist()
pol=df['p_politician'].unique().tolist()
post=df['p_id'].unique().tolist()

#Matrix shape= root/party/party.....pol/pol/pol/...../post/post/.../com/com
total_size=1+len(parties)+len(pol)+len(post)+size_df
shift_size=1+len(parties)+len(pol)+len(post)
matrix = np.identity(total_size,dtype=int)

for i in range(size_df):
    #for each post with have to add 1 to the link party-politician, 1 to the link politician-post, 1 to the link post-comment
    index_party=parties.index(df['p_PARTITO'][i])+1 #add 1 for the root element
    index_pol=pol.index(df['p_politician'][i])+1+len(parties)
    index_post=post.index(df['p_id'][i])+1+len(parties)+len(pol)

    #We fill the 1rst half of the matrix
    matrix[0][index_party]+=1 #add 1 to link root-party
    matrix[index_party][index_pol]+=1 #add 1 to the link party-politician
    matrix[index_pol][i+1+len(parties)+len(pol)]+=1 #1 to the link politician-post
    matrix[index_post][i+shift_size]+=1 #1 to the link post-comment
    #now we fill the other half (lower-left one)
    matrix[index_party][0]+=1
    matrix[index_pol][index_party]+=1
    matrix[i+1+len(parties)+len(pol)][index_pol]+=1 
    matrix[i+shift_size][index_post]+=1

m=np.asmatrix(matrix)
print(m.shape)

(88393, 88393)


In [3]:
import networkx as nx
#First let's try with a small matrix
m_trunc=m[:20,:20]
np.set_printoptions(linewidth=200)
print(m_trunc)
G_trunc = nx.from_numpy_matrix(m_trunc)

[[    1 10245 18760  9544  5296  1692  1659 20519  7224  3236     0     0     0     0     0     0     0     0     0     0]
 [10245     1     0     0     0     0     0     0     0     0   663     0     0  2113     0     0     0     0     0  3499]
 [18760     0     1     0     0     0     0     0     0     0     0   911  2200     0     0     0  1458     0     0     0]
 [ 9544     0     0     1     0     0     0     0     0     0     0     0     0     0  1561  2580     0     0     0     0]
 [ 5296     0     0     0     1     0     0     0     0     0     0     0     0     0     0     0     0  1252     0     0]
 [ 1692     0     0     0     0     1     0     0     0     0     0     0     0     0     0     0     0     0  1249     0]
 [ 1659     0     0     0     0     0     1     0     0     0     0     0     0     0     0     0     0     0     0     0]
 [20519     0     0     0     0     0     0     1     0     0     0     0     0     0     0     0     0     0     0     0]
 [ 7224     0   

In [4]:
import math
def create_attributes_dict_com(pandas_df, index_shift):
    attributes=["p_PARTITO"]
    #attributes=["p_PARTITO","p_politician","p_favoriteCount","p_shareCount","p_replyCount","p_numComments","p_rating","p_topic","p_campagna","c_rating"]
    att_dict = {} #final dict to be returned
    for index in pandas_df.index:
    #for index, row in pandas_df.iterrows():
        #if index not in att_dict: #check seems to make issues...
        temp_dict={}
        for att in attributes:
            #print(att)
            #print(pandas_df[att][0])
            #print(index)
            #print(row)
            #print(pandas_df[att][index])

            #we need to delete the "nan" values
            if(isinstance(pandas_df[att][index], float)):
                if(not math.isnan(pandas_df[att][index])):
                    temp_dict[att]=pandas_df[att][index]
            else:
                temp_dict[att]=pandas_df[att][index]
        temp_dict["label"]=index+shift_size
        temp_dict["type"]="com"
        att_dict[index+index_shift]=temp_dict.copy() #I'm afraid if shallow copy (not sure if usefull though)
    return att_dict

In [5]:
def add_post_pol_party_att(pandas_df, parties, politicians, posts): #param= lists
    att_dict = {}
    for i in range(len(parties)):
        att_dict[i+1]={"label":parties[i], "type":"party"}
    for i in range(len(politicians)):
        att_dict[i+len(parties)+1] = {"label":politicians[i], "type":"politician"}
    for i in range(len(posts)):
        att_dict[i+len(parties)+len(politicians)+1] = {"label":posts[i], "type":"post"}
    return att_dict

In [6]:
#Apply function on real dataframe
G = nx.from_numpy_matrix(m)

In [ ]:
G

In [ ]:
attributes_dict_com=create_attributes_dict_com(df,shift_size)
missing_attributes_dict_com=add_post_pol_party_att(df,parties,pol, post)
attributes_dict_com.update(missing_attributes_dict_com)ss

In [ ]:
nx.set_node_attributes(G, attributes_dict_com)

In [ ]:
nx.write_graphml(G, "com_attributes_networkx_export.graphml") 